In [16]:
!pip install lxml_html_clean

In [14]:
!pip3 install newspaper3k

  Using cached newspaper3k-0.2.8-py3-none-any.whl.metadata (11 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Using cached feedfinder2-0.0.4.tar.gz (3.3 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jieba3k-0.35.1.zip (7.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tinysegmenter-0.3.tar.gz (16 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sgmllib3k-1.0.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using cached newspaper3k-0.2.8-py3-none-any.whl (211 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13566 sha256=ef1d2ced9a16565de4440893168ad7ba135915839193f362971fbea09dd0

In [17]:
import requests
import pandas as pd
from newspaper import Article

# 🔍 네이버 뉴스 검색
def search_naver_news_all(query, target_dates):
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    all_filtered = []
    for start in range(1, 500, 100):
        params = {
            "query": query,
            "display": 100,
            "start": start,
            "sort": "sim"
        }
        try:
            response = requests.get("https://openapi.naver.com/v1/search/news.json", headers=headers, params=params)
            response.raise_for_status()
        except:
            break
        items = response.json().get("items", [])
        for item in items:
            try:
                pubdate = parsedate_to_datetime(item["pubDate"]).date()
                if pubdate in target_dates:
                    all_filtered.append(item["originallink"].replace("amp;", ""))
            except:
                continue
    return all_filtered

# 📄 기사 본문 추출
def extract_article_text(url):
    try:
        article = Article(url, language="ko")
        article.download()
        article.parse()
        return article.text
    except:
        return None

In [19]:
text = extract_article_text('https://search.naver.com/search.naver?ssc=tab.news.all&query=%ED%9A%A1%EB%A0%B9&sm=tab_opt&sort=0&photo=3&field=0&pd=3&ds=2019.06.30&de=2025.06.30&docid=&related=0&mynews=1&office_type=3&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20190630to20250630&is_sug_officeid=0&office_category=1&service_area=0')
print(text)

횡령 ·배임·뇌물 같은 혐의로 수사를 받는 재벌 총수나 정치인들이 검경의 소환 수사를 전후해 몸이 아프다는 ‘감성팔이’ 수법으로 악용하곤 했다. 세인의 뇌리에 오래 남아 있는 게 1997년 정태수 한보그룹 전 회장이다. 외환위기 도화선이 된 비자금과 정·관계 로비 의혹으로 국회 청문회에 출석할 때, 그는...


In [3]:
import requests

def fetch_news_from_webhook(query: str, date: str, news_office_checked: str) -> dict:
    """n8n을 통해 supabase에 뉴스를 저장하는 함수
    
    Args:
        query (str): 검색어
        date (str): 날짜 (YYYY.MM.DD 형식)
        news_office_checked (str): 뉴스사 ID
          1023(조선일보)
          1025(중앙일보)
          1020(동아일보)
          1015(한국경제)
          1009(매일경제)
          1011(서울경제)
        
    Returns:
        dict: 응답 데이터
          sucess : 성공했거나 이미 중복 데이터거나
          retry : 실패 > 재시도
    """
    url = "https://moluvalu.app.n8n.cloud/webhook/3fc6b155-45d8-42cb-b54b-c81bd87ac445"
    params = {
        "query": query,
        "date": date,
        "news_office_checked": news_office_checked
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"요청 중 오류가 발생했습니다: {e}")
        return {}


In [ ]:
from datetime import datetime, timedelta
import time

# 시작 날짜 설정 (2025.06.30)
current_date = datetime(2025, 6, 30)

# 검색어와 뉴스사 리스트 설정
query = "사기"
news_offices = ["1025", "1020"] # 중앙일보, 동아일보

# 과거로 이동하면서 데이터 수집
while current_date.year >= 2025:  # 2025년까지 수집
    
    # 날짜 형식 변환 (YYYY.MM.DD)
    date_str = current_date.strftime("%Y.%m.%d")
    
    # 각 뉴스사별로 데이터 수집
    for news_office in news_offices:
        print(f"수집 중: {date_str} - 뉴스사 {news_office}")
        
        # API 호출 및 재시도 로직
        max_retries = 3
        retry_count = 0
        
        while retry_count < max_retries:
            result = fetch_news_from_webhook(
                query=query,
                date=date_str,
                news_office_checked=news_office
            )
            
            # retry가 있으면 재시도
            if result.get('retry'):
                print(f"재시도 {retry_count + 1}/{max_retries}")
                retry_count += 1
                time.sleep(2)  # 재시도 전 2초 대기
                continue
            else:
                break
                
        # 결과 출력
        print(f"{date_str} {news_office} 결과: {result}")
        
        # API 호출 간 간격 두기 
        time.sleep(2)
    
    # 하루 전으로 이동
    current_date -= timedelta(days=1)
    
print("데이터 수집 완료")


수집 중: 2025.06.30 - 뉴스사 1025
2025.06.30 1025 결과: {'output': 'success\n\n뉴스 데이터가 {"2025.06.30", "아모레퍼시픽", "090430"}에 대해 정상적으로 입력되었습니다. (동일한 source가 이미 존재해도 success만 반환합니다.)'}
수집 중: 2025.06.30 - 뉴스사 1020
2025.06.30 1020 결과: {'output': 'sucess\n\n해당 뉴스 데이터가 정상적으로 입력(또는 이미 존재)하여 sucess를 반환합니다.'}
수집 중: 2025.06.29 - 뉴스사 1025
2025.06.29 1025 결과: {'output': '2025.06.29 에는 데이터가 없습니다'}
수집 중: 2025.06.29 - 뉴스사 1020
2025.06.29 1020 결과: {'output': '알겠습니다. 주어진 요청에 따라 JSON 데이터가 제공되면, 아래와 같은 방식으로 처리하겠습니다.\n\n프로세스 요약:\n\n1. JSON 미제공 시: "2025.06.29 에는 데이터가 없습니다"를 반환합니다.\n2. JSON 제공 시:\n     - 뉴스 데이터에서 상장기업명을 추출합니다.\n     - 상장기업명을 기준으로 종목코드를 찾아냅니다.\n     - 종목코드가 없는 뉴스는 모두 버립니다.\n     - 종목코드가 있는 뉴스만 다음 항목에 맞게 Supabase에 입력합니다:\n         - title → fieldValues3_Field_Value\n         - url(source) → fieldValues4_Field_Value\n         - summary → fieldValues5_Field_Value\n         - stock_code → fieldValues6_Field_Value\n     - 모든 입력이 성공하면 success, 실패 시 retry를 반환합니다.\n     - 이미 같은 source(뉴스 url)가 DB에 있다면 success